### Retrieve data

Define the variables and levels we want to retrieve. Single-level variables ignore the "levels" parameter. Also note that not all variables in the ERA5 dataset are coded with their parameter names as of now. We also take a reduced sample of years in the dataset.

In [ ]:
variables = ['geopotential','2m_temperature']
levels = [500]
years = list(range(1979, 2019))

In [ ]:
import os
os.chdir(os.pardir)
from DLWP.data import ERA5Reanalysis

data_directory = '/lcrc/project/AIEADA-2/cubed_sphere_implementation/Data'
os.makedirs(data_directory, exist_ok=True)
era = ERA5Reanalysis(root_directory=data_directory, file_id='tutorial')
era.set_variables(variables)
era.set_levels(levels)

Download data! Automatically uses multi-processing to retrieve multiple files at a time. Note the parameter `hourly` says we're retrieving only every 3rd hour in the data, which is available hourly. The optional parameter passed to the retrieval package specifies that we want data interpolated to a 2-by-2 latitude-longitude grid.

In [ ]:
ra.retrieve(variables, levels, years=years, hourly=3,
             request_kwargs={'grid': [2., 2.]}, verbose=True, delete_temporary=True)

In [ ]:
era.open()
print(era.Dataset)

Now we use the DLWP.model.Preprocessor tool to generate a new data file ready for use in a DLWP Keras model. Some preliminaries... Note that we assign level "0" to the single-level 2m temperature data. I highly recommend using "pairwise" data processing, which means that each variable is matched to a level pair-wise. The length of the variables and levels lists should be the same. Also note that you only need to specify whole days in the dates. It takes care of the hourly data automatically.

In [ ]:
import pandas as pd
from DLWP.data.era5 import get_short_name

dates = list(pd.date_range('1979-01-01', '2018-12-31', freq='D').to_pydatetime())
variables = get_short_name(variables)
levels = [500,0]
processed_file = '%s/tutorial_z500_t2m_no_scale.nc' % data_directory

In [ ]:
from DLWP.model import Preprocessor

pp = Preprocessor(era, predictor_file=processed_file)
pp.data_to_series(batch_samples=10000, variables=variables, levels=levels, pairwise=True,
                  scale_variables=True, overwrite=True, verbose=True)

In [ ]:
print(pp.data)
pp.data.drop('varlev').to_netcdf(processed_file + '.nocoord')
era.close()
pp.close()

## Remapping training data to the cubed sphere

The novel addition in DLWP-CS is the ability to train convolutional neural networks on data mapped to the cubed sphere. The re-mapping is performed offline from the model training/inference. 

#### Required packages

We use the TempestRemap library for cubed sphere remapping which is available as a pre-compiled conda package. Let's start by installing it.

In [ ]:
%conda install -c conda-forge tempest-remap

In [ ]:
import os
os.chdir(os.pardir)
from DLWP.remap import CubeSphereRemap

data_directory = '/lcrc/project/AIEADA-2/cubed_sphere_implementation/Data'
processed_file = '%s/tutorial_z500_t2m.nc' % data_directory
remapped_file = '%s/tutorial_z500_t2m_CS.nc' % data_directory

csr = CubeSphereRemap()

In [ ]:
csr.generate_offline_maps(lat=91, lon=180, res=48, inverse_lat=True)

Apply the forward map, saving to a temporary file. We specify to operate on the variable `predictors`, which is the only variable in the processed data. TempestRemap is very finicky about metadata in netCDF files, sometimes failing with segmentation faults for no apparent reason. I've found that the most common crash is because it does not like the string coordinate values in the `'varlev'` coordinate. If you used the command in the previous tutorial to produce an extra "nocoord" version of this file, you might *have to* use it here.

In [ ]:
csr.remap(processed_file + '.nocoord', '%s/temp.nc' % data_directory, '--var', 'predictors')

By default, TempestRemap has a 1-dimensional spatial coordinate. We convert the file to 3-dimensional faces (face, height, width). A few other points here:  
- Even if TempestRemap does not crash, it will probably delete the string coordinates, and sometimes the sample time coordinate as well, so it's a good idea to use this feature.  
- We also take advantage of the `chunking` parameter to save data with ideal chunking when using the file for training and evaluating models.

In [ ]:
csr.convert_to_faces('%s/temp.nc' % data_directory, 
                     remapped_file,
                     coord_file=processed_file,
                     chunking={'sample': 1, 'varlev': 1})

In [ ]:
import os
os.remove('%s/temp.nc' % data_directory)

## Training a DLWP-CS model

Now we use the data processed in the previous two notebooks to train a convolutional neural network on weather data mapped to the cubed sphere. We will construct the same convolutional neural network with the cubed sphere as in *Weyn et al. (2020)*, with the exception of having only two variables (Z500 and T2) instead of their four, and without the constant land-sea mask and topography data. This will seem like a fairly involved example but much simpler constructions are also possible using the `DLWPNeuralNet` class instead of the functional Keras API. I also highly recommend having this model train on a GPU with at least 4 GB of video memory.

#### Required packages

No new packages are needed here beyond the main DLWP-CS requirements in the README.

In [ ]:
import os
os.chdir(os.pardir)

root_directory = '/lcrc/project/AIEADA-2/cubed_sphere_implementation/'
predictor_file = os.path.join(root_directory, 'Data', 'tutorial_z500_t2m_CS.nc')
model_file = os.path.join(root_directory, 'dlwp-cs_tutorial')
log_directory = os.path.join(root_directory, 'logs', 'dlwp-cs_tutorial')

In [ ]:
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [ ]:
cnn_model_name = 'unet2'
base_filter_number = 32
min_epochs = 0
max_epochs = 20
patience = 2
batch_size = 32
shuffle = True

In [ ]:
io_selection = {'varlev':['z/500','t2m/0']}
add_solar = True

These parameters govern the time stepping in the model.  
- `io_time_steps`: the number of input/output time steps directly ingested/predicted by the model  
- `integration_steps`: the number of forward sequence steps on which to minimize the loss function of the model  
- `data_interval`: the number of steps in the data file that constitute a "time step." Here we use 2, and the data contains data every 3 hours, so the effective time step is 6 h.
- `loss_by_step`: either None (equal weighting) or a list of weighting factors for the loss function at each integration step.

In [ ]:
io_time_steps = 2
integration_steps = 2
data_interval = 2
loss_by_step = None

In [ ]:
import pandas as pd

train_set = list(pd.date_range('1979-01-01', '2014-12-31 21:00', freq='3H'))
validation_set = list(pd.date_range('2015-01-01', '2016-12-31 21:00', freq='3H'))

In [ ]:
use_mp_optimizer = False

### Create a DLWP model

Since the data generators depend on the model (granted it's an outdated dependency), we make the model instance first.

In [ ]:
from DLWP.model import DLWPFunctional

dlwp = DLWPFunctional(is_convolutional=True, time_dim=io_time_steps)

### Load data and create data generators

DLWP-CS includes powerful data generators that produce batches of training data on-the-fly. This enables them to load only the time series into memory instead of repetitive samples of data. On the downside, it makes reading training data from disk virtually impossibly slow. First, load the data.

In [ ]:
import xarray as xr

data = xr.open_dataset(predictor_file)
train_data = data.sel(sample=train_set)
validation_data = data.sel(sample=validation_set)

Create the training data generator. Here we use the `ArrayDataGenerator` class, which has a nifty pre-processing function to create a single numpy array of data. The `SeriesDataGenerator` is more intuitive and would work equally well. The only reason I don't use the latter is because I thought the overhead when using xarray objects instead of pure numpy might slow things down. It doesn't.

In [ ]:
from DLWP.model.preprocessing import prepare_data_array
from DLWP.model import ArrayDataGenerator

print('Loading data to memory...')
train_array, input_ind, output_ind, sol = prepare_data_array(train_data, input_sel=io_selection,
                                                             output_sel=output, add_insolation=add_solar)
generator = ArrayDataGenerator(dlwp, train_array, rank=3, input_slice=input_ind, output_slice=output_ind,
                               input_time_steps=io_time_steps, output_time_steps=io_time_steps,
                               sequence=integration_steps, interval=data_interval, insolation_array=sol,
                               batch_size=batch_size, shuffle=shuffle, channels_last=True,
                               drop_remainder=True)

In [ ]:
print('Loading validation data to memory...')
val_array, input_ind, output_ind, sol = prepare_data_array(validation_data, input_sel=io_selection,
                                                           output_sel=output, add_insolation=add_solar)
val_generator = ArrayDataGenerator(dlwp, val_array, rank=3, input_slice=input_ind, output_slice=output_ind,
                                   input_time_steps=io_time_steps, output_time_steps=io_time_steps,
                                   sequence=integration_steps, interval=data_interval, insolation_array=sol,
                                   batch_size=batch_size, shuffle=False, channels_last=True)

In [ ]:
from DLWP.model import tf_data_generator

input_names = ['main_input'] + ['solar_%d' % i for i in range(1, integration_steps)]
tf_train_data = tf_data_generator(generator, batch_size=batch_size, input_names=input_names)
tf_val_data = tf_data_generator(val_generator, input_names=input_names)

### Create the CNN model architecture

Now the fun part! Here we create all of the layers that will go into the model. A few notes:  
- The generator produces a list of inputs when `integration_steps` is greater than 1:  
  - main input, including insolation  
  - insolation for step 2  
  - insolation for step 3...  
- We use our custom layers for padding and convolutions on the cubed sphere  
- We can use the Keras 3D layers for operations on the 3D spatial structure of the cubed sphere

In [ ]:
from tensorflow.keras.layers import Input, UpSampling3D, AveragePooling3D, concatenate, ReLU, Reshape, Concatenate, \
    Permute
from DLWP.custom import CubeSpherePadding2D, CubeSphereConv2D

# Some shortcut variables. The generator provides the expected shape of the data.
cs = generator.convolution_shape
cso = generator.output_convolution_shape
input_solar = (integration_steps > 1 and add_solar)

# Define layers. Must be defined outside of model function so we use the same weights at each integration step.
main_input = Input(shape=cs, name='main_input')
if input_solar:
    solar_inputs = [Input(shape=generator.insolation_shape, name='solar_%d' % d) for d in range(1, integration_steps)]
cube_padding_1 = CubeSpherePadding2D(1, data_format='channels_last')
pooling_2 = AveragePooling3D((1, 2, 2), data_format='channels_last')
up_sampling_2 = UpSampling3D((1, 2, 2), data_format='channels_last')
relu = ReLU(negative_slope=0.1, max_value=10.)
conv_kwargs = {
    'dilation_rate': 1,
    'padding': 'valid',
    'activation': 'linear',
    'data_format': 'channels_last'
}
skip_connections = 'unet' in cnn_model_name.lower()
conv_2d_1 = CubeSphereConv2D(base_filter_number, 3, **conv_kwargs)
conv_2d_1_2 = CubeSphereConv2D(base_filter_number, 3, **conv_kwargs)
conv_2d_1_3 = CubeSphereConv2D(base_filter_number, 3, **conv_kwargs)
conv_2d_2 = CubeSphereConv2D(base_filter_number * 2, 3, **conv_kwargs)
conv_2d_2_2 = CubeSphereConv2D(base_filter_number * 2, 3, **conv_kwargs)
conv_2d_2_3 = CubeSphereConv2D(base_filter_number * 2, 3, **conv_kwargs)
conv_2d_3 = CubeSphereConv2D(base_filter_number * 4, 3, **conv_kwargs)
conv_2d_3_2 = CubeSphereConv2D(base_filter_number * 4, 3, **conv_kwargs)
conv_2d_4 = CubeSphereConv2D(base_filter_number * 4 if skip_connections else base_filter_number * 8, 3, **conv_kwargs)
conv_2d_4_2 = CubeSphereConv2D(base_filter_number * 8, 3, **conv_kwargs)
conv_2d_5 = CubeSphereConv2D(base_filter_number * 2 if skip_connections else base_filter_number * 4, 3, **conv_kwargs)
conv_2d_5_2 = CubeSphereConv2D(base_filter_number * 4, 3, **conv_kwargs)
conv_2d_5_3 = CubeSphereConv2D(base_filter_number * 4, 3, **conv_kwargs)
conv_2d_6 = CubeSphereConv2D(base_filter_number if skip_connections else base_filter_number * 2, 3, **conv_kwargs)
conv_2d_6_2 = CubeSphereConv2D(base_filter_number * 2, 3, **conv_kwargs)
conv_2d_6_3 = CubeSphereConv2D(base_filter_number * 2, 3, **conv_kwargs)
conv_2d_7 = CubeSphereConv2D(base_filter_number, 3, **conv_kwargs)
conv_2d_7_2 = CubeSphereConv2D(base_filter_number, 3, **conv_kwargs)
conv_2d_7_3 = CubeSphereConv2D(base_filter_number, 3, **conv_kwargs)
conv_2d_8 = CubeSphereConv2D(cso[-1], 1, name='output', **conv_kwargs)

Now we actually create the output using the functional API. For each operation in the model, we call the appropriate layer on an input tensor `x`. This function performs the operations inside a U-Net, including the skipped connections with concatenation along the channels dimension. This is the sequence of operations to get input data to a prediction, but it is not the whole model, since that one must predict a sequence of 2 (`integration_steps = 2`). That will be next.

In [ ]:
def unet2(x):
    x0 = cube_padding_1(x)
    x0 = relu(conv_2d_1(x0))
    x0 = cube_padding_1(x0)
    x0 = relu(conv_2d_1_2(x0))
    x1 = pooling_2(x0)
    x1 = cube_padding_1(x1)
    x1 = relu(conv_2d_2(x1))
    x1 = cube_padding_1(x1)
    x1 = relu(conv_2d_2_2(x1))
    x2 = pooling_2(x1)
    x2 = cube_padding_1(x2)
    x2 = relu(conv_2d_5_2(x2))
    x2 = cube_padding_1(x2)
    x2 = relu(conv_2d_5(x2))
    x2 = up_sampling_2(x2)
    x = concatenate([x2, x1], axis=-1)
    x = cube_padding_1(x)
    x = relu(conv_2d_6_2(x))
    x = cube_padding_1(x)
    x = relu(conv_2d_6(x))
    x = up_sampling_2(x)
    x = concatenate([x, x0], axis=-1)
    x = cube_padding_1(x)
    x = relu(conv_2d_7(x))
    x = cube_padding_1(x)
    x = relu(conv_2d_7_2(x))
    x = conv_2d_8(x)
    return x

Next we manipulate the result of the CNN back to inputs to the same CNN, add the new insolation input, and pass it through again. This allows us to minimize the loss function at each step of the sequence. Adding the insolation looks complicated because the array includes a time dimension whereas the data inputs are flattened time/variables.

In [ ]:
from tensorflow.keras.layers import Reshape, Concatenate, Permute

def complete_model(x_in):
    outputs = []
    model_function = globals()[cnn_model_name]
    is_seq = isinstance(x_in, (list, tuple))
    xi = x_in[0] if is_seq else x_in
    outputs.append(model_function(xi))
    for step in range(1, integration_steps):
        xo = outputs[step - 1]
        if is_seq and input_solar:
            xo = Reshape(cs[:-1] + (io_time_steps, -1))(xo)
            xo = Concatenate(axis=-1)([xo, Permute((2, 3, 4, 1, 5))(x_in[step])])
            xo = Reshape(cs)(xo)
        outputs.append(model_function(xo))

    return outputs

In [ ]:
from tensorflow.keras.models import Model

if not input_solar:
    inputs = main_input
else:
    inputs = [main_input]
    if input_solar:
        inputs = inputs + solar_inputs
model = Model(inputs=inputs, outputs=complete_model(inputs))

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

loss_function = 'mse'

# Get an optimizer, with mixed precision if requested
opt = Adam()
if use_mp_optimizer:
    opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt)

In [ ]:
dlwp.build_model(model, loss=loss_function, loss_weights=loss_by_step, optimizer=opt, metrics=['mae'])
print(dlwp.base_model.summary())

In [ ]:
from tensorflow.keras.callbacks import History, TensorBoard
from DLWP.custom import EarlyStoppingMin, SaveWeightsOnEpoch, GeneratorEpochEnd

history = History()
early = EarlyStoppingMin(monitor='val_loss' if validation_data is not None else 'loss', min_delta=0.,
                         min_epochs=min_epochs, max_epochs=max_epochs, patience=patience,
                         restore_best_weights=True, verbose=1)
tensorboard = TensorBoard(log_dir=log_directory, update_freq='epoch')

In [ ]:
import time

start_time = time.time()
dlwp.fit_generator(tf_train_data, epochs=max_epochs + 1,
                   verbose=1, validation_data=tf_val_data,
                   callbacks=[history, early, GeneratorEpochEnd(generator)])
end_time = time.time()

In [ ]:
from DLWP.util import save_model

save_model(dlwp, model_file, history=history)
print('Wrote model %s' % model_file)

In [ ]:
print("\nTrain time -- %s seconds --" % (end_time - start_time))

score = dlwp.evaluate(*val_generator.generate([]), verbose=0)
print('Validation loss:', score[0])
print('Other scores:', score[1:])

## Predicting

### Parameters

Some user-specified parameters. The `scale_file` contains the mean and standard of the data (which was dropped in the cubed sphere remapping). The `map_files` were produced by the cubed sphere remapping. We can re-use them here so we don't have to generate them again.

In [ ]:
import os
os.chdir(os.pardir)

root_directory = '/lcrc/project/AIEADA-2/cubed_sphere_implementation/'
predictor_file = os.path.join(root_directory, 'Data', 'tutorial_z500_t2m_CS.nc')
scale_file = os.path.join(root_directory, 'Data', 'tutorial_z500_t2m.nc')

model = os.path.join(root_directory, 'dlwp-cs_tutorial')
map_files = ('map_LL91x180_CS48.nc', 'map_CS48_LL91x180.nc')

In [ ]:
We'll resurrect some parameters from the training tutorial. See that notebook for definitions. Note that we omit `data_interval` because we simply select only every 6 hours from the dataset.

In [ ]:
io_selection = {'varlev': ['z/500', 't2m/0']}
add_solar = True
io_time_steps = 2

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

validation_set = pd.date_range('2016-12-31', '2018-12-31', freq='6H')
validation_set = np.array(validation_set, dtype='datetime64[ns]')

In [ ]:
dates = pd.date_range('2017-01-01', '2018-12-31', freq='7D')
initialization_dates = xr.DataArray(dates)
num_forecast_hours = 5 * 24
dt = 6

In [ ]:
from DLWP.util import load_model, remove_chars, is_channels_last

dlwp = load_model(model)

# File to save the forecast
forecast_file = os.path.join(root_directory, 'forecast_%s.nc' % remove_chars(model.split(os.sep)[-1]))

In [ ]:
all_ds = xr.open_dataset(predictor_file)
predictor_ds = all_ds.sel(sample=validation_set)

In [ ]:
from DLWP.model import SeriesDataGenerator

sequence = dlwp._n_steps if hasattr(dlwp, '_n_steps') and dlwp._n_steps > 1 else None
val_generator = SeriesDataGenerator(dlwp, predictor_ds, rank=3, add_insolation=add_solar,
                                    input_sel=io_selection, output_sel=io_selection,
                                    input_time_steps=io_time_steps, output_time_steps=io_time_steps,
                                    shuffle=False, sequence=sequence, batch_size=32,
                                    load=False, channels_last=is_channels_last(dlwp))

In [ ]:
from DLWP.model import TimeSeriesEstimator

estimator = TimeSeriesEstimator(dlwp, val_generator)

In [ ]:
print('Predicting with model %s...' % model)

# Select the samples from the initialization dates. The first "time" input to the model is actually one time step earlier
samples = np.array([int(np.where(val_generator.ds['sample'] == s)[0]) for s in initialization_dates]) \
    - io_time_steps + 1
time_series = estimator.predict(num_forecast_hours // dt, samples=samples, verbose=1)

In [ ]:
# Transpose if channels_last was used for the model
if is_channels_last(dlwp):
    time_series = time_series.transpose('f_hour', 'time', 'varlev', 'x0', 'x1', 'x2')

In [ ]:
if scale_file is None:
    scale_ds = predictor_ds
else:
    scale_ds = xr.open_dataset(scale_file)
sel_mean = scale_ds['mean'].sel(io_selection)
sel_std = scale_ds['std'].sel(io_selection)
time_series = time_series * sel_std + sel_mean

In [ ]:
from DLWP.verify import add_metadata_to_forecast_cs

fh = np.arange(dt, time_series.shape[0] * dt + 1., dt)
time_series = add_metadata_to_forecast_cs(
    time_series.values,
    fh,
    predictor_ds.sel(**io_selection).sel(sample=initialization_dates)
)

In [ ]:
time_series.drop('varlev').to_netcdf(forecast_file + '.cs')

In [ ]:
from DLWP.remap import CubeSphereRemap

csr = CubeSphereRemap(to_netcdf4=True)
csr.assign_maps(*map_files)
csr.convert_from_faces(forecast_file + '.cs', forecast_file + '.tmp')
csr.inverse_remap(forecast_file + '.tmp', forecast_file, '--var', 'forecast')
os.remove(forecast_file + '.tmp')